In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
import cv2
from scipy.spatial import distance
from keras import layers
from keras import models
from keras import optimizers
from tensorflow import keras
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import backend as K
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
#/kaggle/input/face-mask-lite-dataset/without_mask/
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
with_mask_path = "../input/face-mask-lite-dataset/with_mask"
without_mask_path = "../input/face-mask-lite-dataset/without_mask"

In [3]:
with_mask_img =[]
with_mask_label =[]

for i in os.listdir(with_mask_path):
    img = os.path.join(with_mask_path + '/'+ i)
    with_mask_img.append(img)
    with_mask_label.append('With Mask')
    
without_mask_img =[]
without_mask_label = []

for i in os.listdir(without_mask_path):
    img = os.path.join(without_mask_path + '/'+ i)
    without_mask_img.append(img)
    without_mask_label.append('Without Mask')

In [4]:
with_mask_df = pd.DataFrame()
with_mask_df['image'] = with_mask_img
with_mask_df['target']= with_mask_label

without_mask_df = pd.DataFrame()
without_mask_df['image'] = without_mask_img
without_mask_df['target']= without_mask_label

In [5]:
df = pd.concat([with_mask_df, without_mask_df], axis=0, ignore_index=True)
df=shuffle(df)

In [6]:
plt.figure(figsize=(12,8))
img = load_img(with_mask_df['image'][12])
plt.imshow(img)
plt.title('With Mask', color = 'green', size=14)
plt.show()

In [7]:
plt.figure(figsize=(10,8))
img= load_img(without_mask_df['image'][112])
plt.imshow(img)
plt.title('Without Mask', color='Red', size=14)
plt.show

In [8]:
with_mask_df.size

In [9]:
without_mask_df.size

In [10]:
df

In [11]:
x_train = df.iloc[0:15000,]
x_val = df.iloc[15001:18000,]
x_test = df.iloc[18001:,]

In [12]:
train_ds = ImageDataGenerator(rescale = 1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

train_ds = train_ds.flow_from_dataframe(dataframe=x_train, x_col="image", y_col="target", target_size=(150, 150),
                                        batch_size = 32, class_mode = "binary")

val_ds = ImageDataGenerator(rescale = 1./255)
val_ds = val_ds.flow_from_dataframe(dataframe=x_val, x_col="image", y_col="target", target_size=(150, 150),
                                    batch_size = 32, class_mode = "binary")


test_ds = ImageDataGenerator(rescale = 1./255)
test_ds = test_ds.flow_from_dataframe(dataframe=x_test, x_col = "image", y_col = "target", target_size = (150, 150),
                                          class_mode = "binary", batch_size = 32)

In [13]:
model1 = VGG19(weights='imagenet', include_top=False, input_shape=(128,128,3))

for layer in model1.layers:
    layer.trainable=False
    
model= models.Sequential()
model.add(model1)

model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [14]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [15]:
history = model.fit(train_ds, steps_per_epoch=len(train_ds)//32, epochs = 5,
                    validation_data=val_ds, validation_steps=len(val_ds)//32)

In [16]:
print(history.history.keys())

In [17]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()